In [3]:
import pandas

data_train = pandas.read_csv(filepath_or_buffer='salary-train.csv')
data_test = pandas.read_csv(filepath_or_buffer='salary-test-mini.csv')

In [35]:
from sklearn.feature_extraction import DictVectorizer

data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [37]:
X_train_categ_inversed = enc.inverse_transform(X_train_categ)
X_test_categ_inversed = enc.inverse_transform(X_test_categ)

In [38]:
X_test_categ_inversed

[{'ContractTime=contract': 1.0, 'LocationNormalized=Milton Keynes': 1.0},
 {'ContractTime=permanent': 1.0, 'LocationNormalized=Manchester': 1.0}]

In [40]:
data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [44]:
data_train['FullDescription'] = data_train['FullDescription'].map(lambda item: item.lower())
data_test['FullDescription'] = data_test['FullDescription'].map(lambda item: item.lower())

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=5)

X_train_vec = vectorizer.fit_transform(data_train['FullDescription'])
X_test_vec = vectorizer.transform(data_test['FullDescription'])

print("Done Tfidf")

Done Tfidf


In [62]:
from scipy.sparse import hstack

train_full_data = hstack([X_train_vec, X_train_categ])
test_full_data = hstack([X_test_vec, X_test_categ])

In [67]:
Y = data_train['SalaryNormalized']

In [68]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)
clf.fit(train_full_data, Y) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [69]:
predictions = clf.predict(test_full_data)
predictions

array([ 56554.03693648,  37210.4278811 ])